In [2]:
# coding=utf-8
from os import link, name
from googletrans import Translator
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import pr, print_string
from numpy import add, mod, product
import requests, time ,convert_numbers, random, re
from lxml import html
from requests.api import get
from woocommerce import API
from selectorlib import Extractor
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
wcapi = API(
    url="https://abzarbrand.com",
    consumer_key="ck_bb0f947f73f3d690eb816a960703d84e0ad59723",
    consumer_secret="cs_ad0ce66b5a121e14d7795149425b59f678f4dc98",
    version="wc/v3"
)

In [8]:
def page_links(url):
    # req = requests.get(url, headers={'user-agent': 'Mozilla/5.0 (Windows NT 30.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.83 Safari/537.36'})
    req = requests.get(url)
    page_info = req.text
    my_xpath = html.fromstring(page_info)
    link3 = []
    if 'digikala' in url:
        links = my_xpath.xpath("//a[@class='js-product-url']/@href")
        
        for link in links:
            link3.append("https://www.digikala.com" + link)
    elif 'doctorabzar' in url:
        links = my_xpath.xpath("//div[@class='products__item-info']//@href")
        for i in range(int(len(links)/3)):
            link3.append(links[3*i])
    elif "alborztools" in url:
        link3 = my_xpath.xpath("//div[@class='product-element-top']/a/@href")
    elif "abzarmart" in url:
        link3 = my_xpath.xpath("//h2/a/@href")
    elif "abzarbrand" in url:
        link3 = my_xpath.xpath("//ul[@class='products columns-4']/li/a/@href")
    return link3
def model(name):
    model = ''
    name_words = word_tokenize(name)
    
    ls = []
    for word in name_words:
        encoded_string = word.encode("ascii", "ignore")
        decode_string = encoded_string.decode()
        if len(decode_string) > 0:
            ls.append(decode_string.strip())
    
    for i in name_words:
        if i == 'مدل':
            index_of_m = name_words.index(i)
            index_of_first = ls.index(name_words[index_of_m + 1])
            for j in range(index_of_first, len(ls)):
                model += ls[j] + " "
            return model
        
    if len(ls) > 0:
        model = ls[-1]
    units = ['متر', 'میلی', 'وات','اینچ', 'کیلو', 'کیلوگرم', 'میلیمتر', 'سانتی', 'سانتیمتر', 'گرمی', 'اسب', 'سیلندر', 'لیتری', 'کیلویی', 'میلیمتری', 'آمپر', 'بار', '']
    next_word = int()
    if len(model) > 0:
        if model in name_words:
            length = name_words.index(model) + 1
            if not length == len(name_words):
                next_word = name_words.index(model) + 1
    if next_word > 0:
        for word in units:
            if word == name_words[next_word]:
                model = ''
    return model
def ch(word, n):
    return re.search(r'\b{}\b'.format(word), n)
def prod_cat(data):
    n = data['name']
    data['categories'] = []
    if ch('دریل', n):
        data['categories'].append({'id':'22'})
        if ch('بتن', n):
            data['categories'].append({'id':'48'})
        elif ch('چکشی', n):
            data['categories'].append({'id':'25'})
        elif ch('ستونی', n):
            data['categories'].append({'id':'24'})
        elif ch('سرکج', n):
            data['categories'].append({'id':'49'})
        elif ch('شارژی', n):
            data['categories'].append({'id':'23'})
        elif ch('گیربکسی', n):
            data['categories'].append({'id':'36'})
        elif ch('نمونه', n):
            data['categories'].append({'id':'149'})
        elif ch('هم‌زن', n) or ch('هم زن', n) or ch('همزن', n) or ch('میکسر', n):
            data['categories'].append({'id':'64'})
        else:
            data['categories'].append({'id':'25'})
            data['categories'].append({'id':'36'})
    elif ch('بتن کن', n):
        data['categories'].append({'id':'22'})
        data['categories'].append({'id':'48'})
    elif ch('کمپرسور', n):
        data['categories'].append({'id':'234'})
    elif ch('پولیش', n):
        data['categories'].append({'id':'46'})
    elif ch('وینچ', n):
        data['categories'].append({'id':'481'})
        if ch('دستی', n) or ch('کلاچ', n) or ch('کلاج', n):
            data['categories'].append({'id':'483'})
        elif ch('برقی', n) or ch('تکفاز', n) or ch('تک فاز', n):
            data['categories'].append({'id':'482'})
        elif ch('خودرو', n) or ch('ماشین', n) or ch('افرود', n) or ch('آفرود', n) or ch('ماشینی', n) or ch('خودرویی', n) or ch('آفرودی', n) or ch('افرودی', n):
            data['categories'].append({'id':'484'})
    elif ch('دستگاه جوش', n) or ch('اینورتر', n):
        data['categories'].append({'id':'27'})
    elif ch('دمنده-مکنده', n) or ch('مکنده', n) or ch('بلوور', n) or ch('دمنده', n):
        data['categories'].append({'id':'1298'})
        data['categories'].append({'id':'30'})
        
    elif ch('رنده نجاری', n) or ch('رنده برقی', n):
        data['categories'].append({'id':'60'})
    elif ch('سنباده لرزان', n) or ch('سنباده زن', n) or ch('دستگاه سنباده', n):
        data['categories'].append({'id':'58'})
    elif ch('سنگ رومیزی', n) or ch('سنگ رو میزی', n):
        data['categories'].append({'id':'70'})
    elif ch('شمشاد', n):
        data['categories'].append({'id':'612'})
    elif ch('شیار زن', n) or ch('شیارزن', n):
        data['categories'].append({'id':'67'})
    elif ch('فرز', n):
        data['categories'].append({'id':'56'})
        if ch('آهنگری', n) or ch('اهنگری', n):
            data['categories'].append({'id':'52'})
        elif ch('انگشتی', n):
            data['categories'].append({'id':'55'})
        elif ch('سنگبری', n) or ch('سنگ بری', n):
            data['categories'].append({'id':'53'})
        elif ch('مینیاتوری', n):
            data['categories'].append({'id':'63'})
        elif ch('مینی', n):
            data['categories'].append({'id':'31'})
        elif ch('اهن', n) or ch('آهن', n) or ch('سنگ', n):
            data['categories'].append({'id':'53'})
            data['categories'].append({'id':'52'})
        elif ch('اور فرز', n) or ch('فرز نجاری', n):
            data['categories'].append({'id':'26'})
            
    elif ch('قیچی برقی', n):
        data['categories'].append({'id':'34'})
    elif ch('سشوار', n):
        data['categories'].append({'id':'45'})
    elif ch('کارواش', n):
        data['categories'].append({'id':'37'})
    elif ch('بکس برقی', n) or ch('بکس شارژی', n):
        data['categories'].append({'id':'32'})
    elif ch('بکس', n) and ch('برقی', n):
        data['categories'].append({'id':'32'})
    elif ch('بکس', n) and ch('شارژی', n):
        data['categories'].append({'id':'32'})
    elif ch('چند کاره', n) or ch('همه کاره', n):
        data['categories'].append({'id':'1862'})
    elif ch('اتولوله', n) or ch('اتو لوله', n) or ch('لوله سبز', n):
        data['categories'].append({'id':'68'})
    elif ch('افقی بر', n) or ch('افقی‌بر', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'40'})

    elif ch('اره زنجیری', n) or ch('اره برقی', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'65'})

    elif ch('پروفیل بر', n) or ch('پروفیل‌بر', n) or ch('فلز بز', n) or ch('فلزبر', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'47'})
    elif ch('فارسی', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'50'})
    elif ch('عمودبر', n) or ch('عمود بر', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'62'})
    elif ch('فلکه', n) or ch('اره نواری', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'1861'})
    elif ch('گردبر', n) or ch('گرد بر', n) or ch('دیسکی', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'51'})
    elif ch('اره میزی', n) or ch('رومیزی', n):
        data['categories'].append({'id':'39'})
        data['categories'].append({'id':'54'})
    
    elif ch('دریل پیچگوشتی', n) or ch('دریل پیچ گوشتی', n) or ch('پیچگوشتی شارژی', n) or ch('پیچ گوشتی برقی', n) or ch('پیچگوشتی برقی', n):
        data['categories'].append({'id':'41'})
    elif ch('پیستوله', n):
        if ch('باد', n):
            data['categories'].append({'id':'227'})
        else:
            data['categories'].append({'id':'69'})
    elif ch('جارو', n):
        data['categories'].append({'id':'57'})
    elif ch('چکش تخریب', n) or ch('پیکور', n):
        data['categories'].append({'id':'38'})
    elif ch('مگنت', n) or ch('لیفتر', n):
        data['categories'].append({'id':'487'})
    elif ch('متر لیزری', n):
        data['categories'].append({'id':'202'})
    elif ch('تراز لیزری', n):
        data['categories'].append({'id':'203'})

In [ ]:
def get_prod(link):

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 30_33_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.3663.303 Safari/537.36'}
    # req = requests.get(link, headers=headers)
    req = requests.get(link)
    page_info = req.text
    my_xpath = html.fromstring(page_info)
    data = {}
    # data["status"] = "draft"

    
    if "digikala" in link:
    
        name = my_xpath.xpath("//h1[@class='c-product__title']/text()")[0]
        name = name.replace('\n','')
        data["name"] = str(name)
        # data['name'] = "مجموعه 100 عددی دریل باس با لوازم کامل"
        slug = my_xpath.xpath("//span[@class='c-product__title-en']/text()")
        if len(slug) > 0:
            data["slug"] = str(slug[0])
        else:
            translator = Translator()
            t = translator.translate(data["name"], dest="en")
            data["slug"] = t.text
        price = my_xpath.xpath("//div[@class='c-product__seller-price-pure js-price-value']//text()")
        # if len(price) > 0:
        #     data["regular_price"] = str(int(price[0].replace(',', '')))

        # try:
        att = my_xpath.xpath("//ul[@class='c-params__list']/li//text()")
        keys_index = []
        list_of_dicts = []
        txt = ''
        for i in att:
            if not '\n' in i:
                keys_index.append(att.index(i))
        
        keys_index.append(len(att))
        for i in range(len(keys_index) - 1 ):
            for j in range(keys_index[i] + 1, keys_index[i+1]):
                txt += att[j]
            list_of_dicts.append({'name' : att[keys_index[i]],'options' : txt,'visible': True })
            txt = ''
        random.shuffle(list_of_dicts)
        att_line, text= '', ''

        for d in list_of_dicts:
            if 'سایر' in d['name']:
                text = d['options']
                list_of_dicts.remove(d)
        data["attributes"] = list_of_dicts

        data["images"] = [{"src" : my_xpath.xpath("//img[@class='js-gallery-img']/@data-src")[0], "name" : data["name"], "alt": data["slug"]}]
        pr
        # abzarmart_search_link = 'https://abzarmart.com/catalogsearch/result/?q=' + model(data['name'])
        # req3 = requests.get(abzarmart_search_link, headers=headers)
        # page_info3 = req3.text
        # my_xpath3 = html.fromstring(page_info3)
        # abzarmart_pr_link = my_xpath3.xpath("//h2//a/@href")

        # if len(abzarmart_pr_link) > 0:    
        #     req3 = requests.get(abzarmart_pr_link[0], headers=headers)
        #     page_info3 = req3.text
        #     my_xpath3 = html.fromstring(page_info3)
        #     abzarmart_pr_name = my_xpath3.xpath("//h1//text()")
        #     if model(data['name']) in abzarmart_pr_name[0]:
        #         abzarmart_att = my_xpath3.xpath("//div[@class='std']//li//text()")
        #         random.shuffle(abzarmart_att)
        #         for i in abzarmart_att:
        #             att_line += '⭐ ' + i + '\n'
        data['description'] = ''
        content = my_xpath.xpath("//div[@class='c-mask js-mask']//text()")
        str3 = '<h3><span style="font-size: 33pt; color: #0000ff;">قیمت و خرید {}</span></h3>'.format(data['slug'])
        str3 = 'برای اطلاع از قیمت و خرید {} میتوانید با کارشناسان ابزاربرند تماس بگیرید و از مشاوره رایگان بهره مند شوید.'.format(data['name'])
        if len(att_line) > 0:
            data['description'] += '<h2><span style="color: #0000ff;">ویژگی های {}</span></h2>'.format(data['name']) + att_line
        # if len(content) > 0:
        #     content3 = sent_tokenize(content[0])
        #     if(len(content3) > 3):
        #         data['short_description'] = '<h3><span style="font-size: 33pt; color: #0000ff;">معرفی {}</span></h3>'.format(data['name']) + content3[0] + content3[3] + content3[3]
        #         data['description'] += '<h3><span style="font-size: 33pt; color: #0000ff;">بررسی و مشخصات {}</span></h3>'.format(data['name']) + content[0] + str3 + str3 
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   " ✔️ با ضمانت معتبر【ابزاربرند】")},
            {'key': 'irk_additional_info', 'value' : text.strip()}
        ]
    elif 'abzarmart' in link:
        name = my_xpath.xpath("//h1//text()")[0]
        # name = name.replace('مدل', '')
        # md = model(name)
        # name = name.replace(md, '')

        # name = name + ' مدل ' + md
        data['name'] = name.replace("  ", " ")
        # data['name'] = 'اینورتر جوشکاری 250 آمپر MOSFET وینکس مدل MMA250'
    
        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text

        abzarmart_att = my_xpath.xpath("//div[@class='std']//li//text()")
        att_line = ''
        random.shuffle(abzarmart_att)
        for i in abzarmart_att:
            att_line += '✔ ' + i + '\n'

        data["images"] = []
        images = my_xpath.xpath("//div[@class='product-image-gallery col-xs-12 px-0']/img/@src")
        for img in images:
            data["images"].append({"src" : img, "name" : data["name"], "alt": data["slug"]})
        data["images"].pop(0)
        list_of_dicts = []
        attributies = my_xpath.xpath("//div[contains(@class, 'attribute-name')]/text()[1]")
        contents = my_xpath.xpath("//div[contains(@class, 'attribute-content')]/text()")

        for i in range(len(attributies)):
            list_of_dicts.append({'name' : attributies[i],'options' : contents[i],'visible': True })
        random.shuffle(list_of_dicts)
        data['attributes'] = list_of_dicts

        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   " ✔️ با ضمانت معتبر【ابزاربرند】")},
            {'key': 'irk_additional_info', 'value' : att_line},
            {'key': '_yoast_wpseo_primary_product_brand', 'value': '915'}

        ]
        data['description'] = '<h2><span style="color: #0000ff;">مشخصات {}</span></h2>'.format(data['name']) + att_line

    elif "abzar-yaragh" in link:
        
        name = my_xpath.xpath("//h1[@class='title-product-detail']//text()")
        name = name[0] + name[1]
        data["name"] = str(name)
        slug = my_xpath.xpath("//strong[@class='tags']/text()")
        if len(slug) > 0:
            data['slug'] = slug[0]
        else:
            translator = Translator()
            t = translator.translate(data["name"], dest="en")
            data["slug"] = t.text

        
        # data["images"] = [{"src" : my_xpath.xpath("//img[@class='img-product-detail']/@src")[0], "name" : data["name"], "alt": data["slug"]}]
        
        add_att = my_xpath.xpath("//p[@class='summary-product-detail']/text()")
        x = ''
        att_line = ''
        random.shuffle(add_att)
        for n in add_att:
            att_line += '✔ ' + n + '\n'
        if len(att_line) > 0:
            data['description'] = '<h2><span style="color: #0000ff;">ویژگی های {}</span></h2>'.format(data['name']) + att_line
        keys = my_xpath.xpath("//td[@class='spec-title']/text()")
        vals = my_xpath.xpath("//td[2]/text()")
        list_of_dicts = []
        for i in range(len(keys)):
                list_of_dicts.append({'name' : keys[i],'options' : vals[i],'visible': True })
        random.shuffle(list_of_dicts)
        data['attributes'] = list_of_dicts
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
    
    elif "doctorabzar" in link:

        h1 = my_xpath.xpath("//h1/text()")
        name = h1[0].replace('\t', '')
        mod = model(name)
        name = name.replace(mod,'')
        name = name + 'ادون مدل ' + mod
        slug = link.replace('https://doctorabzar.com/product/','')
        slug = slug.replace('-', ' ')
        slug = slug.replace('/', '')
        if not 'makita' in slug:
            data['slug'] = 'Makita ' + slug
        data['name'] = str(name)
        data['name'] = 'فارسی بر ثابت 355 میلیمتر ماکیتا مدل LS1440'
        img = my_xpath.xpath("//div[@class='woocommerce-product-gallery__image']//@src")
        if len(img) > 0:
            data["images"] = [{"src" : img[0] , "name" : data["name"], "alt": data["slug"]}]
        keys = my_xpath.xpath("//span[@class='technicalspecs-title']//text()")
        values = my_xpath.xpath("//span[@class='technicalspecs-value']//text()")
        for val in values:
            if val == '\n':
                values.remove(val)
        for key in keys:
            if key == '\n':
                keys.remove(key)
        list_of_dicts = []
        for i in range(len(keys)):
            list_of_dicts.append({'name' : keys[i],'options' : values[i],'visible': True })
        data['attributes'] = list_of_dicts
        add_att = my_xpath.xpath("//div[@class='innerContent']/ul/li/text()")
        random.shuffle(add_att)
        att_line = ''
        for i in add_att:
            att_line += '⭐ ' + i + '\n'
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
        short_content = my_xpath.xpath("//div[@class='woocommerce-Tabs-panel woocommerce-Tabs-panel--desc panel entry-content wc-tab']//p/text()")
        cnt = ''
        for i in short_content:
            cnt += i
        str1 =  '<h2><span style="color: #0000ff;">ویژگی های {}</span></h2>'.format(data['name']) + att_line
        str2 = '<h2><span style="color: #0000ff;">خرید و قیمت {}</span></h2>'.format(data['slug']) + 'پشتیبانان ابزاربرند همواره درحال رسیدگی به تماس های شما هستند درصورتی که برای قیمت و خرید {} نیاز به راهنمایی دارید با پشتیبانان ابزاربرند در تماس باشید تا ابزار مناسب کار خود را خریداری نمایید'.format(data['name'])
        data['description'] = cnt + str1 + str2
        if 'دکتر ابزار' in data['description']:
            data['description'] = data['description'].replace('دکتر ابزار', 'ابزاربرند')
    
    
    elif('abzar-online') in link:
        name = my_xpath.xpath("//h1[@class='product-title product_title entry-title']/text()")
        data['name'] = name[0].replace('\n\t', '')
        data['name'] = 'سنباده لرزان 240 وات کرون مدل CT13394'
        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text
        data["images"] = [{"src" : my_xpath.xpath("//meta[@property='og:image']/@content")[0], "name" : data["name"], "alt": data["slug"]}]

        keys = my_xpath.xpath("//th[@class='woocommerce-product-attributes-item__label']/text()")
        values = my_xpath.xpath("//td[@class='woocommerce-product-attributes-item__value']/p/text()")
        list_of_dicts = []
        for i in range(len(keys)):
            list_of_dicts.append({'name':keys[i],'options':values[i],'visible':True})
        random.shuffle(list_of_dicts)
        data["attributes"] = list_of_dicts
        list_of_att = my_xpath.xpath("//div[@class='product-short-description']/ul/li/text()")
        random.shuffle(list_of_att)
        att_line = ''
        for i in list_of_att:
            att_line += '⭐ ' + i + '\n'

        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
        str3 = '<h2><span style="color: #0000ff;">قیمت و خرید {}</span></h2>'.format(data['name']) + 'برای اطلاع از قیمت روز {} و خرید آن میتوانید با مشاوران بخش فروش ابزاربرند تماس بگیرید : 09107108688'.format(data['name'])
        data['description'] = '<h2><span style="color: #0000ff;">مشخصات {}</span></h2>'.format(data['name']) + att_line + str3
    
    
    elif "alborztools" in link:
        data['name'] = str(my_xpath.xpath("//h1/text()")[0])
        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text
        
        resp = my_xpath.xpath("//p[@class='price']//bdi/text()")
        resp3 = my_xpath.xpath("//p[@class='stock out-of-stock']/text()")
        if len(resp) > 0 and len(resp3) == 0:    
            price = int(resp[0].replace(',',''))
            data['regular_price'] = str(int(int(convert_numbers.hindi_to_english(price)) * 0.95))
        images = my_xpath.xpath("//div[@class='product-image-wrap']//@data-thumb")
        img_list = []
        for i in range(len(images)):
            img_list.append({"src" : images[i], "name" : data["slug"], "alt": data["name"]})
            if i > 3:
                break
        
        list_of_att = my_xpath.xpath("//table[@class='woocommerce-product-attributes shop_attributes']//text()")
        l3, l3 = [], []
        for i in list_of_att:
            l3.append(i.strip())

        for i in l3:
            if len(i) > 0:
                l3.append(i)
        list_of_dicts = []
        for i in range(int(len(l3)/3)):
            list_of_dicts.append({'name' : l3[3 * i],'options' : l3[3 * i + 3],'visible': True })
        data["attributes"] = list_of_dicts

        data['images'] = img_list
        list_of_att = my_xpath.xpath("//div[@class='woocommerce-product-details__short-description']/p/text()")
        att_line = ''
        for i in list_of_att:
            i = i.replace('\n', '')
            att_line += '⭐ ' + i + '\n'
        e = Extractor.from_yaml_file("alborztools.yml")
        mahak_text = req.text
        dict_table = e.extract(mahak_text)
        inner_str = ''
        table_str = ''
        if dict_table['keys'] != None:
            for i in range(len(dict_table['keys'])):
                inner_str += """<tr style="height: 38px;">
                <td style="width: 50%; height: 38px; border-style: solid; border-color: #707070;">{}</td>
                <td style="width: 50%; height: 38px; border-style: solid; border-color: #707070;">{}</td>
                </tr>""".format(dict_table['keys'][i],dict_table['values'][i])
            table_str = """
            <table style="height: 38px; width: 50%; border-collapse: collapse; border-style: solid; border-color: #666666;">
            <tbody>
            {}
            </tbody>
            </table>""".format(inner_str)
        
        mahd_link = 'https://mahdeabzar.com/?s=' + 'محک+{}'.format(model(data['name']))
        mahd_req = requests.get(mahd_link, headers=headers)
        m_page_info = mahd_req.text
        m_my_xpath = html.fromstring(m_page_info)
        product_lst = m_my_xpath.xpath("//ul[@class='posts ajax_post']/li/div/a/@href")
        eng_name = []
        content = ''

        if len(product_lst) > 0:
            product_url = product_lst[0]
            mahd3_req = requests.get(product_url, headers=headers)
            m3_page_info = mahd3_req.text
            m3_my_xpath = html.fromstring(m3_page_info)
            eng_name = m3_my_xpath.xpath("//div[@class='product-features']/p//text()")
            cnt = m3_my_xpath.xpath("//div[@class='post-sh dina-more-less']/p//text()")
            for i in cnt:
                content += i + ' '

        if len(eng_name) > 0:
            data['slug'] = eng_name[0]
        # short_des = sent_tokenize(content)
        # for i in range(3):
        #     data['short_description'] += short_des[i]
        str3 = ''
        if len(content) > 3:
            str3 = '<h3><span style="color: #0000ff;">بررسی تخصصی {}</span></h3>'.format(data['name']) + content
        str3 = '<h3><span style="color: #0000ff;">مشخصات {}</span></h3>'.format(data['name']) + att_line + '\n' + table_str
        data['description'] = str3 + str3
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
    elif "iranedon.com" in link:
        data['name'] = my_xpath.xpath("//h1//text()")[0]
        # data['name'] = "کارواش 120 بار ذغالی ادون مدل ED-QJ-18001"
        slug = link.replace('https://iranedon.com/product/','')
        slug = slug.replace('-', ' ')
        data['slug'] = slug.replace('/', '').title()
        if not 'edon' in data['slug']:
            data['slug'] = 'Edon ' + data['slug']

        # data['slug'] = 'cordless blower edon AKM-21'

        e = Extractor.from_yaml_file("iranedon.yml")
        text = req.text
        att_dic = e.extract(text)
        if len(att_dic) > 0:
            cnt = att_dic['att_iranedon']
        else:
            cnt = my_xpath.xpath("//div[@class='wc-tab-inner ']//li//text()")

        list_of_att = []
        att_line = ''
        add_att = []
        for i in cnt:
            if ':' in i:
                index = i.find(':')
                key = ''
                value = ''
                for j in range(index):
                    key += i[j]
                for k in range(index + 1, len(i)):
                    value += i[k]         
                list_of_att.append({'name' : key,'options' : value,'visible': True })
            else:
                add_att.append(i)

        random.shuffle(add_att)
        for n in add_att:
            att_line += '⭐ ' + n + '\n'
        random.shuffle(list_of_att)
        data['attributes'] = list_of_att
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
        if len(att_line) > 0:
            data['description'] = '<h3><span style="color: #0000ff;">مشخصات {}</span></h3>'.format(data['name']) + att_line

    elif 'abzarchi' in link:
        name = my_xpath.xpath("//h1/text()")[0]
        data['name'] = name
        data['name'] = "وینچ خودرو آفرود 5.5 تن کارا 12000lbs"
        data['slug'] = link.replace("https://abzarchi.com/product/", '')
        data['slug'] = data['slug'].replace('-', ' ')
        data['slug'] = data['slug'].replace('/', '').title()
        data['slug'] = 'KARA 12000lbs Car Winch'
        # data['images'] = [{"src" : my_xpath.xpath("//meta[@property='og:image']/@content")[0], "name" : data["slug"], "alt": data["name"]}]
        att_line = ''
        add_att = my_xpath.xpath("//div[@class='single-product-main-description']//li/text()")
        random.shuffle(add_att)
        for n in add_att:
            att_line += '✔ ' + n + '\n'        

        keys = my_xpath.xpath("//li[@class='clearfix']/span/text()")
        values = my_xpath.xpath("//li[@class='clearfix']/span[2]/span/p/text()")
        list_of_dicts = []
        for i in range(len(keys)):
                list_of_dicts.append({'name' : keys[i],'options' : values[i],'visible': True })
        random.shuffle(list_of_dicts)
        data['attributes'] = list_of_dicts

        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
        if len(att_line) > 0:
            data['description'] = '<h2><span style="color: #0000ff;">ویژگی های {}</span></h2>'.format(data['name']) + att_line
    elif "mahdeabzar" in link:
        data['name'] = my_xpath.xpath("//h1/text()")[0]
        if 'DCA' in data['name']:
            data['name'] = data['name'].replace('DCA', 'دی سی ای')
        
        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text
        img = my_xpath.xpath("//div[@class='woocommerce-product-gallery__image flex-active-slide']/a/@href")
        if len(img) > 0:
            data['images'] = [{"src" : img[0], "name" : data["slug"], "alt": data["name"]}]
        
        vals = []

        keys = my_xpath.xpath("//th[@class='woocommerce-product-attributes-item__label']/text()")
        for i in my_xpath.xpath("//td[@class='woocommerce-product-attributes-item__value']//text()"):
            if len(i) > 2:
                vals.append(i)
        list_of_dicts = []
        for i in range(len(keys)):
                list_of_dicts.append({'name' : keys[i],'options' : vals[i],'visible': True })
        random.shuffle(list_of_dicts)
        data['attributes'] = list_of_dicts
        att_line = ''
        add_att = my_xpath.xpath("//div[@class='product-features']//li/text()")
        random.shuffle(add_att)
        for n in add_att:
            att_line += '✔ ' + n + '\n'
        if len(att_line) > 0:
            data['description'] = '<h2><span style="color: #0000ff;">ویژگی های {}</span></h2>'.format(data['name']) + att_line
   

        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            {'key': 'irk_additional_info', 'value' : att_line}
        ]
    elif 'alborzkala' in link:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(link)
        title = driver.find_elements_by_css_selector(".product-title.title")
        data['name'] = title[0].text

        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text

        test = driver.find_elements_by_class_name("technical-list-key")
        print("----------------")
        print(test)
        print("----------------")
        print(type(test), len(test))
        print("-------------------------------")
        for i in test:
            print(i.get_element_by_tag("span").text)
    elif 'torob.com' in link:
        data['name'] = my_xpath.xpath("//h1/text()")[0]
        data['name'] = 'کارواش 60 بار کرون مدل CT42025'
        
        translator = Translator()
        t = translator.translate(data["name"], dest="en")
        data["slug"] = t.text
        img = my_xpath.xpath("//div[@class='jsx-1813026706 gallery']//img/@src")[-1]
        data['images'] = [{"src" : img, "name" : data["slug"], "alt": data["name"]}]

        e = Extractor.from_yaml_file("torob.yml")
        text = req.text
        dict_att = {}
        dict_att['keys'] = []
        dict_att = e.extract(text)
        list_of_dicts = []
        if (type(dict_att['keys']).__name__ != "NoneType"):
            for i in range(len(dict_att['keys'])):
                    list_of_dicts.append({'name' : dict_att['keys'][i],'options' : dict_att['vals'][i],'visible': True })
        random.shuffle(list_of_dicts)
        data['attributes'] = list_of_dicts
        data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
            # {'key': 'irk_additional_info', 'value' : att_line}
        ]
    ###################################
    prod_cat(data)
    # print(data)

    return data

url = 'https://torob.com/p/48e28304-2a9f-4de4-8314-bdbb3e459431/%D8%A7%DB%8C%D9%86%D9%88%D8%B1%D8%AA%D8%B1-300-%D8%A7%D9%85%D9%BE%D8%B1-%D9%85%D8%A7%D8%B3%D9%81%D8%AA-%D8%A7%D9%BE-%D8%A7%D8%B3%D9%BE%D8%B1%DB%8C%D8%AA-%D9%85%D8%AF%D9%84-mma-300/'
src = 'https://storage.torob.com/backend-api/base/images/SU/Q2/SUQ2VTOtUVUAn0Td.jpg'


data = get_prod(url)
data['regular_price'] = '2798000'
# data['sale_price'] = 0.92 * int(data['regular_price'])
# data['images'] = [
#     {"src" : src, "name" : data["slug"], "alt": data["name"]}
# ]

batch = {
    'create': [data],
}
# data['description'] = """
# <h2>مشخصات تبدیل مینی فرز به اره پروفیل بر باس مدل BG-125</h2>
# قابلیت استفاده برای انواع مینی فرز را دارد. لذا با خرید این دستگاه می توانید در موارد و کاربردهای مختلف از آن استفاده نمایید.
# سبک و کم وزن بوده و از این رو حمل و نقل و جابجایی آن به آسانی و سهولت امکان پذیر است.
# جهت تبدیل مینی فرز به اره پروفیل بر و فارسی بر قابل استفاده می باشد.
# دارای گیره می باشد که قابل تغییر زاویه در جهت های مختلف می باشد.
# روی صفحه ی اصلی دستگاه تبدیل مینی فرز به اره پروفیل بر باس جای پیچ وجود دارد که در صورت لزوم به میز کار پیچ شود.
# """
print(wcapi.post("products/batch", batch).json()) 


In [11]:
data = {}
data['name'] = 'مینی فرز دیمردار دسته بلند 1800 وات باس S1M-TD-125B2'
src = 'https://emperor-tools.ir/wp-content/uploads/2020/12/IMG_20201204_232738_781-1.jpg'


data['regular_price'] = '650000'
data['attributes'] = [
    {'name' : ' توان','options' : '1800 وات','visible': True },
    {'name' : 'قطر صفحه','options' : '125 میلی متر','visible': True },
    {'name' : 'سرعت گردش آزاد','options' : '11000 دور در دقیقه','visible': True },
    {'name' : 'اقلام همراه','options' : 'آچار تعویض صفحه ، محافظ صفحه ، دسته جانبی ، دفترچه','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    {'name' : '','options' : '','visible': True },
    
]
text = """

<h2>ویژگی های مینی فرز دیمردار دسته بلند 1800 وات باس S1M-TD-125B2</h2>
قابلیت تنظیم دور
دارای دیمر کنترل سرعت
 دارای موتور کم صدا و با طول عمر دنده های بالا
دارای بلبرینگ ضد غبار
سبک و کم حجم و مناسب جهت امور نصب
قابلیت تعویض سریع و آسان صفحه
قابلیت تنظیم دور
قابلیت استفاده به عنوان پولیش
قابلیت تعویض ذغال بدون باز کردن دستگاه
خوش دست و سبک
سیم پیچ تمام مس
سایز صفحه تا 125 میلیمتر
کیفیت ساخت بسیار بالا
مینی فرز دسته بلند دیمردار باس مدل s1m-td-125b2 برای برش و سابش چوب، سنگ و فلزات مورد استفاده قرار می گیرد باس در این مدل از موتوری با توان 1200 وات استفاده کرده است، این توان سرعت گردش آزاد مینی فرز را تا 11000 دور بر دقیقه را خواهند رساند. از دیگر ویژگی های مینی فرز دسته بلند دیمردار باس مدل s1m-td-125b2 می توان به قابلیت تنظیم دور ،  قابلیت استفاده به عنوان پولیش ، قابلیت تعویض ذغال بدون باز کردن دستگاه ، سیم پیچ تمام مس ،  قفل اسپیندال جهت تعویض آسان صفحه و موتور عایق گرد و خاک و سیستم خنک کنده آن اشاره کرد، تا در هر مکانی بدون نگرانی از دستگاه استفاده نمایید.


وزن مینی فرز دسته بلند دیمردار باس مدل s1m-td-125b2 حدود 2.2 کیلوگرم است که علاوه بر اینکه کاربر را خسته نمی کند دقت در کار برشکاری سابش بالاتر خواهد رفت. باس برای کنترل بیشتر روی مینی فرز همراه با دستگاه دسته کمکی تعبیه کرده است که در هر دو سمت فرز قابل نصب است و باعث کنترل بیشتر شما برروی دستگاه خواهد شد . حداکثر قطر سنگ صفحه فرز استفاده شده در این دستگاه 125 میلی‌متر است، و شما می توانید از صفحه های با قطر کوچکتر نیز بر روی این دستگاه استفاده نمایید. لازم به ذکر است اندازه سوارح مرکزی این دستگاه حدود 14 میلی متر می باشد.


"""
translator = Translator()
t = translator.translate(data["name"], dest="en")
data["slug"] = t.text
data["meta_data"] = [
            {'key' : "_irk_product_en_title", 'value':data["slug"]},
            {'key' : "_yoast_wpseo_title", 'value' : str(data["name"] + "【ابزاربرند】")},
            {'key': '_yoast_wpseo_metadesc', 'value' : str(data["name"] + " ⭐ بهترین قیمت ⭐ " + data["slug"] + " ⚡ " +   "  با ضمانت معتبر【ابزاربرند】 ✔️")},
]
data['images'] = [
    {"src" : src, "name" : data["slug"], "alt": data["name"]}
]
data['description'] = text
prod_cat(data)
batch = {
    'create': [data],
}
# data["status"] = "draft"

print(wcapi.post("products/batch", batch).json()) 


ReadTimeout: HTTPSConnectionPool(host='abzarbrand.com', port=443): Read timed out. (read timeout=5)